- soft dtw divergence : https://rtavenar.github.io/ml4ts_ensai/contents/align/softdtw.html
- contraindre sur la digonale
- comaparer L2 et DTW
- Autoencoder
- VAE
- Detection d'anomalies

In [ ]:
import torch

In [ ]:
def soft_min(list_a, gamma):
    """Softmin function.

    Args:
        list_a (list): list of values
        gamma (float): gamma parameter

    Returns:
        float: softmin value
    """
    assert gamma >= 0, "gamma must be greater than or equal to 0"
    
    # transform list_a to numpy array
    list_a = torch.Tensor(list_a)
    
    if gamma == 0:
        _min = torch.min(list_a)
    else:
        z = -list_a / gamma
        log_sum = torch.max(z) + torch.log(torch.sum(torch.exp(z - max(z))))
        _min = -gamma * log_sum
    return _min


In [ ]:
# create list of values
list_a = [9, 10, 0.5, 1, 2, 3, 4, 100, -1]
gamma = 1

# calculate softmin
_min = soft_min(list_a, gamma)
print(_min)

# calculate softmin with gamma = 0
_min = soft_min(list_a, 0)
print(_min)

In [ ]:
n = 10
m = 10
R = torch.zeros((n, m))
R.shape

In [ ]:
def soft_dtw(x, y, gamma=1.0):
    """Soft Dynamic Time Warping.

    Args:
        x (list): time series 1
        y (list): time series 2
        gamma (float, optional): gamma parameter. Defaults to 1.0.

    Returns:
        float: soft-DTW distance
    """
    # initialize DP matrix
    n = len(x)
    m = len(y)
    R = torch.zeros((n + 1, m + 1))
    R[0, 1:] = float('inf')
    R[1:, 0] = float('inf')
    R[0, 0] = 0.0
    
    for j in range(1, m + 1):
        for i in range(1, n + 1):
            # calculate distance
            cost = (x[i - 1] - y[j - 1])**2
            
            # calculate minimum
            _min = soft_min([R[i - 1, j], R[i, j - 1], R[i - 1, j - 1]], gamma)
            
            # update cell
            R[i, j] = cost + _min
            
    return R[-1, -1], R
    
    

In [ ]:
# create two time series
x = torch.Tensor([-2, 1, 1])
y = torch.Tensor([1, 3, 1, 9, 1])

# calculate soft-DTW distance
gamma = 0.5
dist, R = soft_dtw(x, y, gamma=gamma)
print(dist)

In [ ]:
from tslearn import metrics
metrics.soft_dtw(x, y, gamma=gamma)